In [ ]:
# install syft if we are on google colab
%load_ext autoreload
%autoreload 2
import syft as sy
print(f"You're running syft version: {sy.__version__}")
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import preprocessing
from syft.core.tensor.autodp.gamma_tensor import GammaTensor
from tqdm import tqdm
import time

In [ ]:
domain_client = sy.login(email='info@openmined.org', password='changethis',port = 8081)

In [ ]:
data = domain_client.datasets[-1]

X_train = data["train_images"]
Y_train = data["train_labels"]

X_dev = data["train_images"]
Y_dev = data["train_labels"]

m,n = X_train.public_shape

X_train = X_train.T
X_train = X_train *(1/ 255.0)

X_dev = X_dev.T
X_dev = X_dev *(1/ 255.0)

In [ ]:
def init_params(input_size: int):
    print(f"Using input size: {input_size}")
    W1 = np.random.rand(6, input_size) - 0.5
    b1 = np.random.rand(6, 1) - 0.5
    W2 = np.random.rand(6, 6) - 0.5
    b2 = np.random.rand(6, 1) - 0.5
    return W1, b1, W2, b2

def ReLU(Z):
    return Z*(Z>0)

def softmax(Z):
    exp_cache = Z.exp()
    inv =  (exp_cache.sum().reciprocal())
    
    A = exp_cache * inv
    return A
    
def forward_prop(W1, b1, W2, b2, X):    
    Z1 = X.__rmatmul__(W1) + b1
    A1 = ReLU(Z1)
    Z2 = A1.__rmatmul__(W2) + b2
    A2 = softmax(Z2) 
    return Z1, A1, Z2, A2


def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size),Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = Y.one_hot()
    dZ2 = A2 - one_hot_Y
    dW2 = dZ2@(A1.T) * (1/m)
    db2 = dZ2.sum() * (1/m)
    dZ1 = dZ2.__rmatmul__(W2.T) * ReLU_deriv(Z1)
    dW1 = dZ1@(X.T) * (1/m)
    db1 = dZ1.sum()*(1/m)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = (dW1 * alpha - W1) * -1
    b1 = (db1 * alpha - b1) * -1
    W2 = (dW2 * alpha - W2) * -1
    b2 = (db2 * alpha - b2) * -1
    return W1, b1, W2, b2


def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params(X.public_shape[0])
    print("[INFO]: Starting training!\n")
    for i in tqdm(range(iterations)):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)

        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        # if i % 40 == 0:
        #     print("Predicition will be availabe in the next iteration.....Thank you for your support.")
        #     # predictions = get_predictions(A2)
        #     # print("Iteration: ", i, " Accuracy : " ,get_accuracy(predictions, Y))

    return W1, b1, W2, b2

In [ ]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10, 1)

In [ ]:
for ptr in [W1, b1, W2, b2]:
    ptr.block_with_timeout(60)

In [ ]:
print(W1.exists)
print(b1.exists)
print(W2.exists)
print(b2.exists)

In [ ]:
W1_O = W1.get_copy()
print(type((W1_O.child)))

In [ ]:
b1_O = b1.get_copy()
print(type((b1_O.child)))

In [ ]:
W2_O = W2.get_copy()
print(type((W2_O.child)))

In [ ]:
b2_O = b2.get_copy()
print(type((b2_O.child)))